In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
from malaya_speech.train.model import denoiser, stft
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.




In [4]:
i = tf.placeholder(tf.float32, [None, 1])
model = denoiser.Model(i, logging = True, stride = 4)

Tensor("Pad_1:0", shape=(?, 88405, 1), dtype=float32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Tensor("concat_2:0", shape=(?, ?, 1), dtype=float32)
Tensor("concat_2:0", shape=(?, ?, 1), dtype=float32)
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Tensor("sequential/conv1d_1/mul:0", shape=(?, ?, 48), dtype=float32)
Tensor("sequential_2/conv1d_4/mul:0", shape=(?, ?, 96), dtype=float32)
Tensor("sequential_4/conv1d_7/mul:0", shape=(?, ?, 192), dtype=float32)
Tensor("sequential_6/conv1d_10/mul:0", shape=(?, ?, 384), dtype=float32)
x Tensor("sequential_8/conv1d_13/mul:0", shape=(?, ?, 768), dtype=float32)
Tensor("dense/BiasAdd:0", shape=(?, ?, 768), dtype=float32)
Tensor("sequential_9/Conv1DTranspose/strided_slice:0", shape=(?, ?, 384), dtype=float32)
Tensor("sequential_7

In [5]:
logits = model.logits[:tf.shape(i)[0]]

In [6]:
# https://github.com/facebookresearch/denoiser/blob/master/denoiser/stft_loss.py#L109
stft_loss = stft.loss.MultiResolutionSTFT(factor_sc = 0.1, factor_mag = 0.1)

In [7]:
l1 = tf.reduce_mean(tf.abs(i - logits))

In [8]:
# https://github.com/facebookresearch/denoiser/blob/master/denoiser/solver.py
sc_loss, mag_loss = stft_loss(tf.expand_dims(i[:,0], 0), tf.expand_dims(logits[:,0], 0))
loss = l1 + sc_loss + mag_loss

In [9]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [10]:
y, sr = malaya_speech.load('../speech/7021-79759-0004.flac', sr = 44100)
y = y[:sr * 10]
len(y), len(y) / sr

(441000, 10.0)

In [11]:
%%time

r = sess.run(model.logits, feed_dict = {i: np.expand_dims(y, -1)})

CPU times: user 12.9 s, sys: 3.6 s, total: 16.5 s
Wall time: 3.11 s


In [12]:
%%time

sess.run(loss, feed_dict = {i: np.expand_dims(y, -1)})

CPU times: user 14.2 s, sys: 3.73 s, total: 17.9 s
Wall time: 3.21 s


array([0.46095908], dtype=float32)

In [13]:
# saver = tf.train.Saver(tf.trainable_variables())
# saver.save(sess, 'test/model.ckpt')

In [14]:
tf.trainable_variables()

[<tf.Variable 'sequential/conv1d/kernel:0' shape=(8, 1, 48) dtype=float32>,
 <tf.Variable 'sequential/conv1d/bias:0' shape=(48,) dtype=float32>,
 <tf.Variable 'sequential/conv1d_1/kernel:0' shape=(1, 48, 96) dtype=float32>,
 <tf.Variable 'sequential/conv1d_1/bias:0' shape=(96,) dtype=float32>,
 <tf.Variable 'conv1d/kernel:0' shape=(1, 48, 48) dtype=float32_ref>,
 <tf.Variable 'conv1d/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'conv1d_1/kernel:0' shape=(1, 48, 48) dtype=float32_ref>,
 <tf.Variable 'conv1d_1/bias:0' shape=(48,) dtype=float32_ref>,
 <tf.Variable 'sequential_2/conv1d_3/kernel:0' shape=(8, 48, 96) dtype=float32>,
 <tf.Variable 'sequential_2/conv1d_3/bias:0' shape=(96,) dtype=float32>,
 <tf.Variable 'sequential_2/conv1d_4/kernel:0' shape=(1, 96, 192) dtype=float32>,
 <tf.Variable 'sequential_2/conv1d_4/bias:0' shape=(192,) dtype=float32>,
 <tf.Variable 'conv1d_2/kernel:0' shape=(1, 96, 96) dtype=float32_ref>,
 <tf.Variable 'conv1d_2/bias:0' shape=(96,) dtype=float